In [0]:
'''
AiCore Pinterest Data Pipeline Project
Stream Pinterest data from Kinesis; clean and analyse the data with PySpark.
This code is intended to run in a Databricks notebook.
Author: Kristina Gorkovskaya
Date: 2023-11-28
'''

from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark
import tempfile
import urllib

In [0]:
%run /Users/kristina.gorkovskaya@gmail.com/schemas

In [0]:
# Replace with your AWS IAM user id
user_id ='0ec858bf1407'

# Read the CSV file (credentials) to spark dataframe
file_type = "csv"
first_row_is_header = "true"
delimiter = ","
aws_keys_df = spark.read.format(file_type) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df \
  .where(col('User name')=='databricks-user') \
  .select('Access key ID') \
  .collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df \
  .where(col('User name')=='databricks-user') \
  .select('Secret access key') \
  .collect()[0]['Secret access key']

# Encode the secret key; safe="" means every char will be encoded
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

def read_stream(stream_name_suffix: str) -> pyspark.sql.dataframe.DataFrame:
  '''Read Kinesis stream.'''

  stream_name = f'streaming-{user_id}-{stream_name_suffix}'
  print(f'Reading stream {stream_name}...')
  df = spark \
    .readStream \
    .format('kinesis') \
    .option('streamName', stream_name) \
    .option('initialPosition','earliest') \
    .option('region','us-east-1') \
    .option('awsAccessKey', ACCESS_KEY) \
    .option('awsSecretKey', SECRET_KEY) \
    .load()
  return df

def parse_stream(stream_name_suffix: str, schemas: list) -> pyspark.sql.dataframe.DataFrame:
  '''Read Kinesis stream and apply a schema to parse JSON data to create one column per attribute.'''
  df = read_stream(stream_name_suffix)
  schema = schemas[stream_name_suffix]
  df = df.selectExpr("cast (data as STRING) my_json_data").select(from_json("my_json_data", schema).alias("tmp")).select("tmp.*")
  return df

In [0]:
#############################################################################################################
# TASK 1: Clean the df_pin DataFrame
#############################################################################################################

df_pin = parse_stream('pin', schemas)

# (1) Replace nans and empty strings with None
df_pin = df_pin.replace(float('nan'), None).replace("", None)

# (2) Perform the necessary transformations on the follower_count to ensure every entry is a number.
# Create a multiplier column
# Assumptions: a suffix of "k" in follower_count means a multiplier of 1,000; a suffix of "M" means 10^6
df_pin = df_pin.withColumn('follower_count_multiplier', 
                           when(df_pin.follower_count.like('%k'), 1000).
                           when(df_pin.follower_count.like('%M'), 1000000).
                           otherwise(1))

# Parse numeric data from follower_count and apply multiplier to the parsed values
df_pin = df_pin.withColumn('follower_count_numeric',
                           when(df_pin.follower_count == 'User Info Error', None).
                           otherwise(regexp_replace('follower_count', '[^0-9]+', '')).
                           cast(IntegerType()) * df_pin.follower_count_multiplier)

# Remove intermediate columns
df_pin = df_pin.withColumn('follower_count', df_pin.follower_count_numeric)
df_pin = df_pin.drop('follower_count_numeric', 'follower_count_multiplier')

# (3) Ensure that each column containing numeric data has a numeric data type
numeric_cols = ['downloaded', 'index']
for c in numeric_cols:
    df_pin = df_pin.withColumn(c, df_pin[c].cast(IntegerType()))

# (4) Clean the data in the save_location column to include only the save location path
df_pin = df_pin.withColumn('save_location', regexp_replace('save_location', 'Local save in ', ''))

# (5) Rename the index column to ind, and reorder the columns.
cols = [    
    'ind',
    'unique_id',
    'title',
    'description',
    'follower_count',
    'poster_name',
    'tag_list',
    'is_image_or_video',
    'image_src',
    'save_location',
    'category'
    ]
df_pin = df_pin.withColumnRenamed('index', 'ind').select(cols)

Reading stream streaming-0ec858bf1407-pin...

In [0]:
#############################################################################################################
# TASK 2: Clean the df_geo (geolocation) DataFrame
#############################################################################################################

df_geo = parse_stream('geo', schemas)

# (1) Create a new column coordinates that contains an array based on the latitude and longitude columns
df_geo = df_geo.withColumn('coordinates', array('latitude', 'longitude'))

# (2) Drop the latitude and longitude columns from the DataFrame
df_geo = df_geo.drop('latitude', 'longitude')

# (3) Convert the timestamp column from a string to a timestamp data type
df_geo = df_geo.withColumn('timestamp', to_timestamp('timestamp'))

# (4) Reorder columns
df_geo = df_geo.select('ind', 'country', 'coordinates', 'timestamp')

Reading stream streaming-0ec858bf1407-geo...

In [0]:
#############################################################################################################
# TASK 3: Clean the df_user DataFrame
#############################################################################################################

df_user = parse_stream('user', schemas)

# (1) Create a new column user_name that concatenates the information found in the first_name and last_name columns.
# Trim and normalize whitespace.
df_user = df_user.withColumn('user_name', regexp_replace(trim(concat_ws(' ', 'first_name', 'last_name')), '\s+', ' '))

# (2) Drop the first_name and last_name columns from the DataFrame
df_user = df_user.drop('first_name', 'last_name')

# (3) Convert the date_joined column from a string to a timestamp data type
df_user = df_user.withColumn('date_joined', to_timestamp('date_joined'))

# (4) Reorder columns.
df_user = df_user.select('ind', 'user_name', 'age', 'date_joined')

Reading stream streaming-0ec858bf1407-user...

In [0]:
#############################################################################################################
# FINAL TASK: Write transformed data to Databricks delta tables. 
#############################################################################################################

# Clear delta tables
#%fs rm -r /user/hive/warehouse/0ec858bf1407_geo_table
#%fs rm -r /user/hive/warehouse/0ec858bf1407_pin_table
#%fs rm -r /user/hive/warehouse/0ec858bf1407_user_table

In [0]:
def write_stream(df: pyspark.sql.dataframe.DataFrame, delta_table_name: str) -> None:
  '''Write streaming data to delta table. '''
  with tempfile.TemporaryDirectory() as d:
    df.writeStream \
      .format('delta') \
      .outputMode('append') \
      .option("checkpointLocation", d) \
      .option("startingOffsets", "earliest") \
      .table(delta_table_name)  

write_stream(df_geo, f'{user_id}_geo_table')

In [0]:
write_stream(df_pin, f'{user_id}_pin_table')

In [0]:
write_stream(df_user, f'{user_id}_user_table')